In [12]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/ML/innovation_lab

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/MyDrive/ML/innovation_lab'
/content/drive/MyDrive/ML/innovation_lab


In [13]:
"""
DATA EXTRACTION FROM REVIEW TEXT
"""

import os
import re
import pandas as pd
import numpy as np
from typing import Sized
import json
import matplotlib.pyplot as plt
import pdb

review_df=pd.read_csv('/content/drive/MyDrive/ML/innovation_lab/golden_info/review_hedge_score.csv')
review_df

,review_id,meta_decision,review,sentence count,word count,avg. sentence length,avg. word length,vocab length,hedge words,non-hedge words,hedge ratio,sections covered (out of 14),aspects covered (out of 9),section distribution,aspect distribution,section score,aspect score,hedge_score
0,ICLR2018-B11bwYgfM-R1,NaN,The idea of using cross-task transfer performa...,12,242,20.166667,5.223176,121,2,240,0.008333,4,2,"{'ABS': 0, 'INT': 0, 'RWK': 2, 'PDI': 1, 'DAT'...","{'APR': 0, 'NOV': 3, 'IMP': 0, 'CMP': 0, 'PNF'...",0.286886,0.223556,0.053719
1,ICLR2018-B11bwYgfM-R2,NaN,This paper proposes a method for multitask and...,19,402,25.125000,5.311653,158,2,400,0.005000,5,2,"{'ABS': 0, 'INT': 0, 'RWK': 0, 'PDI': 1, 'DAT'...","{'APR': 0, 'NOV': 0, 'IMP': 0, 'CMP': 0, 'PNF'...",0.361318,0.225954,0.007389
2,ICLR2018-B11bwYgfM-R3,NaN,The authors propose techniques for multitask a...,35,763,19.564103,4.860955,278,11,752,0.014628,8,4,"{'ABS': 0, 'INT': 1, 'RWK': 3, 'PDI': 5, 'DAT'...","{'APR': 0, 'NOV': 1, 'IMP': 0, 'CMP': 4, 'PNF'...",0.577550,0.459101,0.037859
3,ICLR2018-B12Js_yRb-R1,Accept,Summary: - This paper proposes a hand-designe...,30,447,14.900000,5.240196,210,1,446,0.002242,11,7,"{'ABS': 0, 'INT': 2, 'RWK': 10, 'PDI': 3, 'DAT...","{'APR': 0, 'NOV': 0, 'IMP': 1, 'CMP': 5, 'PNF'...",0.789854,0.785466,0.014925
4,ICLR2018-B12Js_yRb-R2,Accept,Summary - This paper mainly focuses on a coun...,12,243,20.250000,5.757447,125,0,243,0.000000,8,3,"{'ABS': 0, 'INT': 2, 'RWK': 1, 'PDI': 3, 'DAT'...","{'APR': 0, 'NOV': 0, 'IMP': 0, 'CMP': 1, 'PNF'...",0.572586,0.336443,0.023256
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1317,ICLR2018-SysEexbRb-R2,Accept,This paper studies the critical points of shal...,23,456,20.727273,5.265700,194,3,453,0.006623,10,4,"{'ABS': 2, 'INT': 3, 'RWK': 1, 'PDI': 1, 'DAT'...","{'APR': 1, 'NOV': 0, 'IMP': 0, 'CMP': 1, 'PNF'...",0.717566,0.453774,0.036403
1318,ICLR2018-SysEexbRb-R3,Accept,This paper mainly focuses on the square loss f...,16,323,14.681818,5.738562,137,1,322,0.003106,8,7,"{'ABS': 0, 'INT': 1, 'RWK': 3, 'PDI': 0, 'DAT'...","{'APR': 0, 'NOV': 1, 'IMP': 2, 'CMP': 4, 'PNF'...",0.573514,0.780257,0.015198
1319,ICLR2018-Syt0r4bRZ-R1,Reject,Summary: the paper proposes a tree2tree archit...,20,323,14.681818,5.124590,167,2,321,0.006231,9,5,"{'ABS': 0, 'INT': 1, 'RWK': 2, 'PDI': 0, 'DAT'...","{'APR': 0, 'NOV': 1, 'IMP': 0, 'CMP': 2, 'PNF'...",0.646486,0.561458,0.023952
1320,ICLR2018-Syt0r4bRZ-R2,Reject,This paper proposes a tree-to-tree model aimin...,19,281,15.611111,4.949458,139,4,277,0.014440,9,6,"{'ABS': 0, 'INT': 1, 'RWK': 4, 'PDI': 1, 'DAT'...","{'APR': 1, 'NOV': 1, 'IMP': 0, 'CMP': 3, 'PNF'...",0.644999,0.671141,0.020833


In [14]:
#VADER SENTIMENT

!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

def senti(sentence):
  sid_obj = SentimentIntensityAnalyzer()
  sentiment_dict = sid_obj.polarity_scores(sentence)
  return sentiment_dict['compound']

senti_score_fin = []

rev_text_list=list(review_df['review'])

for count in range(review_df.shape[0]):
  if rev_text_list[count] is not np.nan:
    senti_score_fin.append(senti(rev_text_list[count]))
  else:
    senti_score_fin.append(0)

review_df=pd.concat([review_df,pd.DataFrame({'vader compound sentiment':senti_score_fin})],axis=1)
review_df

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


,review_id,meta_decision,review,sentence count,word count,avg. sentence length,avg. word length,vocab length,hedge words,non-hedge words,hedge ratio,sections covered (out of 14),aspects covered (out of 9),section distribution,aspect distribution,section score,aspect score,hedge_score,vader compound sentiment
0,ICLR2018-B11bwYgfM-R1,NaN,The idea of using cross-task transfer performa...,12,242,20.166667,5.223176,121,2,240,0.008333,4,2,"{'ABS': 0, 'INT': 0, 'RWK': 2, 'PDI': 1, 'DAT'...","{'APR': 0, 'NOV': 3, 'IMP': 0, 'CMP': 0, 'PNF'...",0.286886,0.223556,0.053719,-0.7295
1,ICLR2018-B11bwYgfM-R2,NaN,This paper proposes a method for multitask and...,19,402,25.125000,5.311653,158,2,400,0.005000,5,2,"{'ABS': 0, 'INT': 0, 'RWK': 0, 'PDI': 1, 'DAT'...","{'APR': 0, 'NOV': 0, 'IMP': 0, 'CMP': 0, 'PNF'...",0.361318,0.225954,0.007389,-0.2271
2,ICLR2018-B11bwYgfM-R3,NaN,The authors propose techniques for multitask a...,35,763,19.564103,4.860955,278,11,752,0.014628,8,4,"{'ABS': 0, 'INT': 1, 'RWK': 3, 'PDI': 5, 'DAT'...","{'APR': 0, 'NOV': 1, 'IMP': 0, 'CMP': 4, 'PNF'...",0.577550,0.459101,0.037859,-0.7209
3,ICLR2018-B12Js_yRb-R1,Accept,Summary: - This paper proposes a hand-designe...,30,447,14.900000,5.240196,210,1,446,0.002242,11,7,"{'ABS': 0, 'INT': 2, 'RWK': 10, 'PDI': 3, 'DAT...","{'APR': 0, 'NOV': 0, 'IMP': 1, 'CMP': 5, 'PNF'...",0.789854,0.785466,0.014925,0.9741
4,ICLR2018-B12Js_yRb-R2,Accept,Summary - This paper mainly focuses on a coun...,12,243,20.250000,5.757447,125,0,243,0.000000,8,3,"{'ABS': 0, 'INT': 2, 'RWK': 1, 'PDI': 3, 'DAT'...","{'APR': 0, 'NOV': 0, 'IMP': 0, 'CMP': 1, 'PNF'...",0.572586,0.336443,0.023256,0.9816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1317,ICLR2018-SysEexbRb-R2,Accept,This paper studies the critical points of shal...,23,456,20.727273,5.265700,194,3,453,0.006623,10,4,"{'ABS': 2, 'INT': 3, 'RWK': 1, 'PDI': 1, 'DAT'...","{'APR': 1, 'NOV': 0, 'IMP': 0, 'CMP': 1, 'PNF'...",0.717566,0.453774,0.036403,0.9958
1318,ICLR2018-SysEexbRb-R3,Accept,This paper mainly focuses on the square loss f...,16,323,14.681818,5.738562,137,1,322,0.003106,8,7,"{'ABS': 0, 'INT': 1, 'RWK': 3, 'PDI': 0, 'DAT'...","{'APR': 0, 'NOV': 1, 'IMP': 2, 'CMP': 4, 'PNF'...",0.573514,0.780257,0.015198,-0.7655
1319,ICLR2018-Syt0r4bRZ-R1,Reject,Summary: the paper proposes a tree2tree archit...,20,323,14.681818,5.124590,167,2,321,0.006231,9,5,"{'ABS': 0, 'INT': 1, 'RWK': 2, 'PDI': 0, 'DAT'...","{'APR': 0, 'NOV': 1, 'IMP': 0, 'CMP': 2, 'PNF'...",0.646486,0.561458,0.023952,-0.0403
1320,ICLR2018-Syt0r4bRZ-R2,Reject,This paper proposes a tree-to-tree model aimin...,19,281,15.611111,4.949458,139,4,277,0.014440,9,6,"{'ABS': 0, 'INT': 1, 'RWK': 4, 'PDI': 1, 'DAT'...","{'APR': 1, 'NOV': 1, 'IMP': 0, 'CMP': 3, 'PNF'...",0.644999,0.671141,0.020833,-0.9454


In [15]:
#POS FEATURES

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

def pos_count(text):
  noun_count=0
  adj_count=0
  verb_count=0
  adverb_count=0
  if text is np.nan:
    text=''
  tokens = nltk.word_tokenize(text.lower())
  text = nltk.Text(tokens)
  tags = nltk.pos_tag(text)
  from collections import Counter
  counts = dict(Counter(tag for word,tag in tags))
  for i in list(counts.keys()):
    if i[0]=='N':
      noun_count+=counts[i]
    if i[0]=='J':
      adj_count+=counts[i]
    if i[0]=='V':
      verb_count+=counts[i]
    if i[0]=='R':
      adverb_count+=counts[i]
  return (noun_count, adj_count, verb_count, adverb_count)

noun_count_list=[]
adj_count_list=[]
verb_count_list=[]
adverb_count_list=[]
for row in range(len(review_df['review'])):
  pos_tup=pos_count(review_df['review'][row])
  noun_count_list.append(pos_tup[0])
  adj_count_list.append(pos_tup[1])
  verb_count_list.append(pos_tup[2])
  adverb_count_list.append(pos_tup[3])
review_df['noun_count']=noun_count_list
review_df['adj_count']=adj_count_list
review_df['verb_count']=verb_count_list
review_df['adverb_count']=adverb_count_list

# print("POS counts (noun, adjective, verb, adverb): ({}, {}, {}, {})".format(review_df['noun_count'][n],review_df['adj_count'][n],review_df['verb_count'][n],review_df['adverb_count'][n]))
review_df

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,review_id,meta_decision,review,sentence count,word count,avg. sentence length,avg. word length,vocab length,hedge words,non-hedge words,...,section distribution,aspect distribution,section score,aspect score,hedge_score,vader compound sentiment,noun_count,adj_count,verb_count,adverb_count
0,ICLR2018-B11bwYgfM-R1,NaN,The idea of using cross-task transfer performa...,12,242,20.166667,5.223176,121,2,240,...,"{'ABS': 0, 'INT': 0, 'RWK': 2, 'PDI': 1, 'DAT'...","{'APR': 0, 'NOV': 3, 'IMP': 0, 'CMP': 0, 'PNF'...",0.286886,0.223556,0.053719,-0.7295,77,34,38,11
1,ICLR2018-B11bwYgfM-R2,NaN,This paper proposes a method for multitask and...,19,402,25.125000,5.311653,158,2,400,...,"{'ABS': 0, 'INT': 0, 'RWK': 0, 'PDI': 1, 'DAT'...","{'APR': 0, 'NOV': 0, 'IMP': 0, 'CMP': 0, 'PNF'...",0.361318,0.225954,0.007389,-0.2271,133,44,76,27
2,ICLR2018-B11bwYgfM-R3,NaN,The authors propose techniques for multitask a...,35,763,19.564103,4.860955,278,11,752,...,"{'ABS': 0, 'INT': 1, 'RWK': 3, 'PDI': 5, 'DAT'...","{'APR': 0, 'NOV': 1, 'IMP': 0, 'CMP': 4, 'PNF'...",0.577550,0.459101,0.037859,-0.7209,216,72,136,49
3,ICLR2018-B12Js_yRb-R1,Accept,Summary: - This paper proposes a hand-designe...,30,447,14.900000,5.240196,210,1,446,...,"{'ABS': 0, 'INT': 2, 'RWK': 10, 'PDI': 3, 'DAT...","{'APR': 0, 'NOV': 0, 'IMP': 1, 'CMP': 5, 'PNF'...",0.789854,0.785466,0.014925,0.9741,129,51,99,32
4,ICLR2018-B12Js_yRb-R2,Accept,Summary - This paper mainly focuses on a coun...,12,243,20.250000,5.757447,125,0,243,...,"{'ABS': 0, 'INT': 2, 'RWK': 1, 'PDI': 3, 'DAT'...","{'APR': 0, 'NOV': 0, 'IMP': 0, 'CMP': 1, 'PNF'...",0.572586,0.336443,0.023256,0.9816,83,24,46,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1317,ICLR2018-SysEexbRb-R2,Accept,This paper studies the critical points of shal...,23,456,20.727273,5.265700,194,3,453,...,"{'ABS': 2, 'INT': 3, 'RWK': 1, 'PDI': 1, 'DAT'...","{'APR': 1, 'NOV': 0, 'IMP': 0, 'CMP': 1, 'PNF'...",0.717566,0.453774,0.036403,0.9958,132,62,67,22
1318,ICLR2018-SysEexbRb-R3,Accept,This paper mainly focuses on the square loss f...,16,323,14.681818,5.738562,137,1,322,...,"{'ABS': 0, 'INT': 1, 'RWK': 3, 'PDI': 0, 'DAT'...","{'APR': 0, 'NOV': 1, 'IMP': 2, 'CMP': 4, 'PNF'...",0.573514,0.780257,0.015198,-0.7655,95,63,41,14
1319,ICLR2018-Syt0r4bRZ-R1,Reject,Summary: the paper proposes a tree2tree archit...,20,323,14.681818,5.124590,167,2,321,...,"{'ABS': 0, 'INT': 1, 'RWK': 2, 'PDI': 0, 'DAT'...","{'APR': 0, 'NOV': 1, 'IMP': 0, 'CMP': 2, 'PNF'...",0.646486,0.561458,0.023952,-0.0403,92,41,60,22
1320,ICLR2018-Syt0r4bRZ-R2,Reject,This paper proposes a tree-to-tree model aimin...,19,281,15.611111,4.949458,139,4,277,...,"{'ABS': 0, 'INT': 1, 'RWK': 4, 'PDI': 1, 'DAT'...","{'APR': 1, 'NOV': 1, 'IMP': 0, 'CMP': 3, 'PNF'...",0.644999,0.671141,0.020833,-0.9454,73,35,58,22


In [16]:
#INFO SCORE

#lists:
info_score_fin = []

# weights:
import json
f = open('/content/drive/MyDrive/ML/innovation_lab/golden_info/weights.json')
data = json.load(f)
s_wt_dic,a_wt_dic=list(data.values())[0],list(data.values())[1]
sec_wt=list(s_wt_dic.values())
asp_wt=list(a_wt_dic.values())
   
sec_dist_list=list(review_df['section distribution'])
asp_dist_list=list(review_df['aspect distribution'])

for count in range(review_df.shape[0]):
  sec_dist_dic=json.loads((sec_dist_list[count]).replace("'",'"'))
  asp_dist_dic=json.loads((asp_dist_list[count]).replace("'",'"'))
  sec_dist=list(sec_dist_dic.values())[:]
  asp_dist=list(asp_dist_dic.values())[:]
  sec_count=0
  asp_count=0
  for i in range(len(sec_dist)):
    if sec_dist[i]>0:
      sec_count+=1
      sec_dist[i]=(sec_dist[i]-1)*(sec_wt[i])
  for i in range(len(asp_dist)):
    if asp_dist[i]>0:
      asp_count+=1
      asp_dist[i]=(asp_dist[i]-1)*(asp_wt[i])

  s_len=0
  a_len=0
  for i in sec_dist:
    s_len+=i
  for i in asp_dist:
    a_len+=i

  section_score=((sec_count+s_len)/len(sec_dist))
  aspect_score=((asp_count+a_len)/len(asp_dist))
  # if section_score>=1:
  #   section_score=1
  # if aspect_score>=1:
  #   aspect_score=1
  nhcount=list(review_df['non-hedge words'])[count]
  hscore=list(review_df['hedge_score'])[count]

  if nhcount>0:
    info_score=section_score/((pow(np.e,1-aspect_score))*(pow(np.e,hscore)))
  else:
    info_score=0
  info_score_fin.append(info_score)

review_df=pd.concat([review_df,pd.DataFrame({'new info score':info_score_fin})],axis=1)
review_df.to_csv('/content/drive/MyDrive/ML/innovation_lab/golden_info/Review_Dataset_fin.csv',index=False)
review_df

,review_id,meta_decision,review,sentence count,word count,avg. sentence length,avg. word length,vocab length,hedge words,non-hedge words,...,aspect distribution,section score,aspect score,hedge_score,vader compound sentiment,noun_count,adj_count,verb_count,adverb_count,new info score
0,ICLR2018-B11bwYgfM-R1,NaN,The idea of using cross-task transfer performa...,12,242,20.166667,5.223176,121,2,240,...,"{'APR': 0, 'NOV': 3, 'IMP': 0, 'CMP': 0, 'PNF'...",0.286886,0.223556,0.053719,-0.7295,77,34,38,11,0.125076
1,ICLR2018-B11bwYgfM-R2,NaN,This paper proposes a method for multitask and...,19,402,25.125000,5.311653,158,2,400,...,"{'APR': 0, 'NOV': 0, 'IMP': 0, 'CMP': 0, 'PNF'...",0.361318,0.225954,0.007389,-0.2271,133,44,76,27,0.165393
2,ICLR2018-B11bwYgfM-R3,NaN,The authors propose techniques for multitask a...,35,763,19.564103,4.860955,278,11,752,...,"{'APR': 0, 'NOV': 1, 'IMP': 0, 'CMP': 4, 'PNF'...",0.577550,0.459101,0.037859,-0.7209,216,72,136,49,0.323771
3,ICLR2018-B12Js_yRb-R1,Accept,Summary: - This paper proposes a hand-designe...,30,447,14.900000,5.240196,210,1,446,...,"{'APR': 0, 'NOV': 0, 'IMP': 1, 'CMP': 5, 'PNF'...",0.789854,0.785466,0.014925,0.9741,129,51,99,32,0.627904
4,ICLR2018-B12Js_yRb-R2,Accept,Summary - This paper mainly focuses on a coun...,12,243,20.250000,5.757447,125,0,243,...,"{'APR': 0, 'NOV': 0, 'IMP': 0, 'CMP': 1, 'PNF'...",0.572586,0.336443,0.023256,0.9816,83,24,46,11,0.288113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1317,ICLR2018-SysEexbRb-R2,Accept,This paper studies the critical points of shal...,23,456,20.727273,5.265700,194,3,453,...,"{'APR': 1, 'NOV': 0, 'IMP': 0, 'CMP': 1, 'PNF'...",0.717566,0.453774,0.036403,0.9958,132,62,67,22,0.400710
1318,ICLR2018-SysEexbRb-R3,Accept,This paper mainly focuses on the square loss f...,16,323,14.681818,5.738562,137,1,322,...,"{'APR': 0, 'NOV': 1, 'IMP': 2, 'CMP': 4, 'PNF'...",0.573514,0.780257,0.015198,-0.7655,95,63,41,14,0.453430
1319,ICLR2018-Syt0r4bRZ-R1,Reject,Summary: the paper proposes a tree2tree archit...,20,323,14.681818,5.124590,167,2,321,...,"{'APR': 0, 'NOV': 1, 'IMP': 0, 'CMP': 2, 'PNF'...",0.646486,0.561458,0.023952,-0.0403,92,41,60,22,0.407099
1320,ICLR2018-Syt0r4bRZ-R2,Reject,This paper proposes a tree-to-tree model aimin...,19,281,15.611111,4.949458,139,4,277,...,"{'APR': 1, 'NOV': 1, 'IMP': 0, 'CMP': 3, 'PNF'...",0.644999,0.671141,0.020833,-0.9454,73,35,58,22,0.454663


In [17]:
# meta_feat=['review_id', 'meta_decision', 'review']

# count_feat=['sentence count', 'word count', 'avg. sentence length', 'avg. word length', 'vocab length']
# hedgelist_feat=['hedge words', 'non-hedge words','hedge ratio']
# hedgepeer_feat=['hedge_score']
# senti_feat=['vader compound sentiment']
# pos_feat=['noun_count', 'adj_count', 'verb_count', 'adverb_count']
# secasp_coverage_feat=['sections covered (out of 14)', 'aspects covered (out of 9)']

# secasp_dist_dic=['section distribution', 'aspect distribution']
# scores=['section score', 'aspect score']
# info_score=['new info score']

In [18]:
# #FEATURES MATRIX CONTAINING INDIVIDUAL COVERAGE OF EVERY SECTION AND ASPECT, FOR FUTURE WORK

# import pandas as pd

# sec_dist=list(review_df['section distribution'])
# asp_dist=list(review_df['aspect distribution'])

# ABS_list=[]
# INT_list=[]
# RWK_list=[]
# PDI_list=[]
# DAT_list=[]
# MET_list=[]
# EXP_list=[]
# RES_list=[]
# TNF_list=[]
# ANA_list=[]
# FWK_list=[]
# OAL_list=[]
# BIB_list=[]
# EXT_list=[]
# sec_list=[ABS_list, INT_list, RWK_list, PDI_list, DAT_list, MET_list, EXP_list, RES_list, TNF_list, ANA_list, FWK_list, OAL_list, BIB_list, EXT_list]

# APR_list=[]
# NOV_list=[]
# IMP_list=[]
# CMP_list=[]
# PNF_list=[]
# REC_list=[]
# EMP_list=[]
# SUB_list=[]
# CLA_list=[]
# asp_list=[APR_list, NOV_list, IMP_list, CMP_list, PNF_list, REC_list, EMP_list, SUB_list, CLA_list]

# for i in sec_dist:
#   i=i.replace("\'", "\"")
#   i=json.loads(i)
#   sec_dic=list(i.values())
#   for j in range(len(sec_list)):
#     sec_list[j].append(sec_dic[j])

# for i in asp_dist:
#   i=i.replace("\'", "\"")
#   i=json.loads(i)
#   asp_dic=list(i.values())
#   for j in range(len(asp_list)):
#     asp_list[j].append(asp_dic[j])

# labels_dict={'ABS':sec_list[0],'INT':sec_list[1],'RWK':sec_list[2],'PDI':sec_list[3],'DAT':sec_list[4],'MET':sec_list[5],'EXP':sec_list[6],
#              'RES':sec_list[7],'TNF':sec_list[8],'ANA':sec_list[9],'FWK':sec_list[10],'OAL':sec_list[11],'BIB':sec_list[12],'EXT':sec_list[13],
#              'APR':asp_list[0],'NOV':asp_list[1],'IMP':asp_list[2],'CMP':asp_list[3],'PNF':asp_list[4],'REC':asp_list[5],'EMP':asp_list[6],'SUB':asp_list[7],'CLA':asp_list[8]}
# temp_df=pd.DataFrame(labels_dict)

# features_df=review_df[meta_feat+count_feat+hedgelist_feat+hedgepeer_feat+senti_feat+pos_feat+secasp_coverage_feat+secasp_dist_dic].copy()
# features_df=pd.concat([features_df, temp_df, review_df[scores+info_score].copy()], axis=1)
# features_df.to_csv('/content/drive/MyDrive/ML/innovation_lab/golden_info/Informativeness_features.csv', index=False)

# features_df


,review_id,meta_decision,review,sentence count,word count,avg. sentence length,avg. word length,vocab length,hedge words,non-hedge words,...,IMP,CMP,PNF,REC,EMP,SUB,CLA,section score,aspect score,new info score
0,ICLR2018-B11bwYgfM-R1,NaN,The idea of using cross-task transfer performa...,12,242,20.166667,5.223176,121,2,240,...,0,0,0,0,3,0,0,0.286886,0.223556,0.125076
1,ICLR2018-B11bwYgfM-R2,NaN,This paper proposes a method for multitask and...,19,402,25.125000,5.311653,158,2,400,...,0,0,0,0,7,1,0,0.361318,0.225954,0.165393
2,ICLR2018-B11bwYgfM-R3,NaN,The authors propose techniques for multitask a...,35,763,19.564103,4.860955,278,11,752,...,0,4,1,0,24,0,0,0.577550,0.459101,0.323771
3,ICLR2018-B12Js_yRb-R1,Accept,Summary: - This paper proposes a hand-designe...,30,447,14.900000,5.240196,210,1,446,...,1,5,1,3,11,10,1,0.789854,0.785466,0.627904
4,ICLR2018-B12Js_yRb-R2,Accept,Summary - This paper mainly focuses on a coun...,12,243,20.250000,5.757447,125,0,243,...,0,1,0,0,6,0,1,0.572586,0.336443,0.288113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1317,ICLR2018-SysEexbRb-R2,Accept,This paper studies the critical points of shal...,23,456,20.727273,5.265700,194,3,453,...,0,1,1,0,16,0,0,0.717566,0.453774,0.400710
1318,ICLR2018-SysEexbRb-R3,Accept,This paper mainly focuses on the square loss f...,16,323,14.681818,5.738562,137,1,322,...,2,4,1,0,4,3,1,0.573514,0.780257,0.453430
1319,ICLR2018-Syt0r4bRZ-R1,Reject,Summary: the paper proposes a tree2tree archit...,20,323,14.681818,5.124590,167,2,321,...,0,2,0,0,10,2,2,0.646486,0.561458,0.407099
1320,ICLR2018-Syt0r4bRZ-R2,Reject,This paper proposes a tree-to-tree model aimin...,19,281,15.611111,4.949458,139,4,277,...,0,3,0,0,7,5,2,0.644999,0.671141,0.454663
